In [ ]:
!pip install numpy scikit-surprise pandas

In [56]:
from collections import defaultdict

from surprise import Dataset, SVD

import pandas as pd

from surprise import Dataset, NormalPredictor, Reader
from surprise.model_selection import cross_validate

from surprise import KNNBasic
#from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
# from surprise import SVD, accuracy
from surprise import accuracy

In [57]:
def get_top_n(predictions, n=10):
    # https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-get-the-top-n-recommendations-for-each-user 
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [58]:
%%bash
set -x
ls -lah ../gdown_output/
ls -lah .

+ ls -lah ../gdown_output/


total 4.3G
drwxr-xr-x 3 user user 4.0K Feb 12 12:49 .
drwxr-xr-x 8 user user 4.0K Feb 12 11:51 ..
drwxr-xr-x 2 user user 4.0K Feb 12 12:42 children
-rw-r--r-- 1 user user  27M Feb  6 15:51 goodreads_books_poetry.json.gz
-rw-r--r-- 1 user user 4.1G Feb  6 15:54 goodreads_interactions.csv
-rw-r--r-- 1 user user   93 Feb  6 15:51 goodreads_interactions.csv.sha256sum
-rw-r--r-- 1 user user 145M Feb  6 15:51 goodreads_interactions_poetry.json.gz
-rw-r--r-- 1 user user  48M Feb  6 15:51 goodreads_reviews_poetry.json.gz


+ ls -lah .


total 1.7M
drwxr-xr-x 5 user user 4.0K Feb 12 14:45 .
drwxr-xr-x 8 user user 4.0K Feb 12 11:51 ..
-rw-r--r-- 1 user user  29K Feb 12 14:45 approach1.ipynb
-rw-r--r-- 1 user user  69K Feb  6 15:36 distributions.ipynb
-rw-r--r-- 1 user user  25K Feb  6 15:41 download.ipynb
-rw-r--r-- 1 user user 2.5K Feb  6 15:36 gdrive_id.csv
drwxr-xr-x 8 user user 4.0K Feb 12 14:42 .git
-rw-r--r-- 1 user user 1.2K Feb  6 15:36 .gitignore
drwxr-xr-x 2 user user 4.0K Feb 12 12:13 .ipynb_checkpoints
-rw-r--r-- 1 user user  12K Feb  6 15:36 LICENSE
-rw-r--r-- 1 user user 2.3K Feb  6 15:36 README.md
-rw-r--r-- 1 user user  14K Feb  6 15:36 reviews.ipynb
-rw-r--r-- 1 user user  17K Feb  6 15:36 samples.ipynb
-rw-r--r-- 1 user user  21K Feb  6 15:36 statistics.ipynb
drwxr-xr-x 2 user user 4.0K Feb 12 12:11 .virtual_documents
-rw-r--r-- 1 user user 1.5M Feb 12 14:45 xaa


In [59]:
%%bash
#gunzip -k ../gdown_output/goodreads_interactions_poetry.json.gz
# split -l 100000 interactions.csv
# head -n 100000 orig.csv  > xaa
head -n 100000  ../gdown_output/goodreads_interactions.csv  > xaa

In [60]:
%%bash
set -x
ls -la xaa
du -h xaa
wc -l xaa
sha1sum xaa
head xaa

+ ls -la xaa


-rw-r--r-- 1 user user 1511415 Feb 12 14:45 xaa


+ du -h xaa


1.5M	xaa


+ wc -l xaa


100000 xaa


+ sha1sum xaa


c144902b17aeb896897f30ac3ebe59e73e2d6558  xaa


+ head xaa


user_id,book_id,is_read,rating,is_reviewed
0,948,1,5,0
0,947,1,5,1
0,946,1,5,0
0,945,1,5,0
0,944,1,5,0
0,943,1,5,0
0,942,1,5,0
0,941,1,5,0
0,940,1,5,0


In [61]:
'''xaa: file with downsampled dataset -- e.g. first 100_000 lines of csv file'''
df = pd.read_csv("xaa")
df = df.drop(columns=["is_read", "is_reviewed"])
df

,user_id,book_id,rating
0,0,948,5
1,0,947,5
2,0,946,5
3,0,945,5
4,0,944,5
...,...,...,...
99994,227,17543,2
99995,227,882,3
99996,227,59168,3
99997,227,24428,3


In [62]:
# Creation of the dataframe. Column names are irrelevant.
ratings_dict = {
    "itemID": [1, 1, 1, 2, 2],
    "userID": [9, 32, 2, 45, "user_foo"],
    "rating": [3, 2, 4, 3, 1],
}
df = pd.DataFrame(ratings_dict)

# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[["userID", "itemID", "rating"]], reader)

# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(NormalPredictor(), data, cv=2)

{'test_rmse': array([0.62300564, 1.52252484]),
 'test_mae': array([0.50268469, 1.50429093]),
 'fit_time': (0.00014710426330566406, 3.7670135498046875e-05),
 'test_time': (5.984306335449219e-05, 2.86102294921875e-05)}

In [63]:
algo = KNNBasic()

In [64]:
reader = Reader()
data = Dataset.load_from_df(df[["user_id", "book_id", "rating"]], reader)

KeyError: "['user_id', 'book_id'] not in index"

In [65]:
trainset, testset = train_test_split(data, test_size=0.25)

In [66]:
algo = SVD()
algo.fit(trainset)

In [67]:
predictions = algo.test(testset)

In [68]:
accuracy.rmse(predictions)

RMSE: 1.8933


1.893344621575743

In [69]:
algo.predict

<bound method AlgoBase.predict of <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7b8b0b1794f0>>

In [70]:
get_top_n(predictions, n=10)[220]

[]